In [1]:
import re
import numpy as np
import pandas as pd
import os, gc
from tqdm.auto import tqdm
from datetime import datetime, timezone, timedelta

import torch
from torch.utils.data import DataLoader,Dataset

c:\Users\kangs\miniconda3\envs\newenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import json
from collections import OrderedDict

In [4]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

In [5]:
import datasets
import transformers

In [6]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

# 토크나이저와 모델 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')

model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

model_1 = BartForConditionalGeneration.from_pretrained('D:/VS_code/1st')

model_3 = BartForConditionalGeneration.from_pretrained('D:/VS_code/3st')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [7]:
# DEVICE 초기화
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"DEVICE:{DEVICE}")

# model을 평가모드로 바꿔서 학습이 일어나지 않도록 하고, DEVICE에 전달하기
model.eval()
model = model.to(DEVICE)
model_1 = model_1.to(DEVICE)
model_3 = model_3.to(DEVICE)

DEVICE:cuda


In [8]:
# 원문 데이터 전처리 하는 부분
def token(text):
    text = re.sub('[^!-~ 가-힣]', '', text)
    text = text.replace('  ', '')
    text = tokenizer.encode(text)

    return text

In [9]:
# 전처리된 데이터 요약하는 부분
def summary(text, model, length = 1.0, num = 4, device = DEVICE):
    text = torch.tensor([text]).to(device)
    text = model.generate(text,  length_penalty=length,  num_beams=num,  max_length=512,  eos_token_id=1)
    text = tokenizer.decode(text.squeeze().tolist(), skip_special_tokens=True)

    return text

In [10]:
%cd D:/VS_code/KBS_news

D:\VS_code\KBS_news


In [11]:
####################################################### KBS 스크랩 #######################################################

In [12]:
# KBS에서 긁어올 뉴스 분야
KBS_field = ['정치', '경제', '사회', '문화', 'IT·과학', '국제', '재난·환경', '생활·건강', '스포츠', '연예', '날씨', '이슈']

In [13]:
# KBS 뉴스 크롤링 후 json 파일 형태로 return 하는 함수
def KBS_data(KBS_number):
  # 입력받은 KBS 뉴스 번호로 url에서 html 코드를 그대로 불러오는 부분
  url = "https://news.kbs.co.kr/news/view.do?ncd="+str(KBS_number)
  result = requests.get(url)
  obj = BeautifulSoup(result.content, "html.parser")

  # 불러온 url에 뉴스가 없을 경우(등록일이 없을 경우 뉴스가 없음) None으로 return 하기
  if (obj.find('li', {'class' : 'time'}) == None):
    return None
  # 문장 토큰 갯수가 모델 입력 최대 수인 1026개 이상일 경우 건너뛰기
  text = token((((obj.find('div', {'class' : 'view_con_text'})).text).split('\n'))[1].replace('\t', ''))
  if (len(text) > 1026):
    return None

  # 우리가 필요한 뉴스 분야만 긁어오는 조건문
  if ((((obj.find('li', {'class' : 'time'})).text).split('\n'))[-2] in KBS_field):
    data = OrderedDict()
    
    # 순서대로 썸네일, 신문사, 제목, 링크, 등록시간, 분야, 작가, 요약문(단문, 중문, 장문)을 dictionary 형태로 저장
    data['image'] = obj.head.find('meta', {'itemprop' : 'image'}).get('content')
    data['company'] = 'KBS'
    data['title'] = obj.head.find('meta', {'name' : 'title'}).get('content')
    data['url'] = obj.head.find('meta', {'name' : 'url'}).get('content')
    data['time'] = (((obj.find('li', {'class' : 'time'})).text).split('\n'))[2][3:-3]
    data['field'] = (((obj.find('li', {'class' : 'time'})).text).split('\n'))[-2]
    # 작가의 경우 기사에 따라 존재하지 않을 때도 있음(대표적으로 날씨 뉴스) 이 때 오류 방지를 위해 None 값이 저장되도록 설정
    if (obj.find('p', {'class' : "name"}) == None):
      data['writer'] = None
    else:
      data['writer'] = (((obj.find('p', {'class' : "name"})).text).split('\n'))[1][:-2]
    # 긁어온 데이터에서 원문에 해당하는 부분을 바로 summary 함수에 넣어서 요약문을 저장하도록 설정
    data['text_short'] = summary(text, model_1, length=2.0, num=8)
    data['text_middle'] = summary(text, model, length=2.0, num=8)
    data['text_long'] = summary(text, model_3)

    return data

In [27]:
####################################################### 실시간 스크랩 #######################################################

In [16]:
# KBS 실시간 웹크롤링 코드
KBS_number = 7680371
while True:
  # data 스크랩 해오기
  data = KBS_data(KBS_number)

  # data 체크
  if (data == None):
    # 스크랩한 번호부터 100까지 순차적으로 더하면서 사이 번호를 건너뛰고 생성된 뉴스가 있는지 체크
    for count in range(100):
      data = KBS_data(KBS_number + count)
      # 만약 생성된 뉴스가 있다면 KBS_number 값을 해당 뉴스 번호로 저장하고, 반복문 탈출
      if (data != None):
        KBS_number = KBS_number + count
        break
    # 시스템 과열 방지를 위해 1분간 휴식시키는 부분, 작동 체크를 위한 선 출력
    print('Sleep_KBS_N : ', KBS_number)
    #time.sleep(60)
  else: 
    # data가 null 값이 아니라면, json 파일로 뽑아내서 저장함
    # 파일명은 '(KBS_number)_KBS_news.json' 형태로 저장 (KBS_number는 숫자)
    with open(str(KBS_number) + '_KBS_news.json', 'w', encoding='utf-8') as make_file:
      json.dump(data, make_file, ensure_ascii=False, indent='\t')
    print('---------------------------------')
    print('KBS_N : ', KBS_number)
    # 이미 한번 체크한 번호니까 다음 번호부터 체크하도록 +1 하기
    KBS_number += 1

Sleep_KBS_N :  7678692
Sleep_KBS_N :  7678696
---------------------------------
KBS_N :  7678696
Sleep_KBS_N :  7678701
---------------------------------
KBS_N :  7678701
Sleep_KBS_N :  7678704
---------------------------------
KBS_N :  7678704
---------------------------------
KBS_N :  7678705
Sleep_KBS_N :  7678712
---------------------------------
KBS_N :  7678712
Sleep_KBS_N :  7678715
Sleep_KBS_N :  7678717
Sleep_KBS_N :  7678719
---------------------------------
KBS_N :  7678719
---------------------------------
KBS_N :  7678720
---------------------------------
KBS_N :  7678721
Sleep_KBS_N :  7678726
---------------------------------
KBS_N :  7678726
Sleep_KBS_N :  7678749
Sleep_KBS_N :  7678765
Sleep_KBS_N :  7678767
---------------------------------
KBS_N :  7678767
---------------------------------
KBS_N :  7678768
---------------------------------
KBS_N :  7678769
Sleep_KBS_N :  7678774
---------------------------------
KBS_N :  7678774
---------------------------------
KBS_

KeyboardInterrupt: 